# Open Model Zoo Tools: Model Downloader, Converter and Info Dumper

This demo shows how to download models from Open Model Zoo and convert them to IR models. 


The [Model Downloader](https://github.com/openvinotoolkit/open_model_zoo/blob/master/tools/downloader/README.md) downloads models from the Open Model Zoo.

By default, the code in this notebook downloads all [Open Model Zoo](https://github.com/openvinotoolkit/open_model_zoo/) models with the specified precision. This allows you to run all demo's. You can change this by removing the `--all` option from the `downloader_command` and adding either a `--name` or a `--list` option. The `--name` option accepts wildcards. For example: to download all `bert` models, add `"--name"`, `"bert*"` to the `downloader_command`. The `--list` option is useful if you want to run a specific demo. All Open Model Zoo demos include a `models.lst` file that contains the models that are compatible with that demo. Add `"--list"`, `"/path/to/models.lst"` to only download models in that list. See the [Open Model Zoo Model Downloader documentation](https://github.com/openvinotoolkit/open_model_zoo/blob/master/tools/downloader/README.md) for more details.

<div class="alert alert-info" style="color:black"><i>
<b>Note: </b>It will take a while to download and convert all the models. If you have a slow internet connection or only want to download a subset of models, change the downloader_command and specify a specific model to download.</div> 

## Imports

In [ ]:
import os.path
import subprocess
import sys

from openvino.inference_engine import IECore

## Settings

Set the file and directory paths. By default, this demo notebook downloads all models from the Open Model Zoo to a folder `open_model_zoo_models` in your `$HOME` directory. On Windows, the $HOME directory is usually `c:\users\username`, on Linux `/home/username`.  If you want to change the folder, change `base_model_dir`. The Model Converter converts OMZ models that are not in IR format to this format. 

You can change the following settings:

* `base_model_dir`: Models will be downloaded into the `intel` and `public` folders in this directory
* `omz_cache_dir`: Cache folder for Open Model Zoo. 
* `precision`: If specified, download only models with this precision. For inference on CPU, FP16 and FP32 gives the same result.


In [ ]:
open_model_zoo_path = os.path.dirname(
    os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(os.curdir)))))
)

base_model_dir = os.path.expanduser("~/open_model_zoo_models")
omz_cache_dir = os.path.expanduser("~/open_model_zoo_cache")
precision = "FP16"

## Download models from Open Model Zoo



Specify and display the Model Downloader command to run

In [ ]:
model_downloader = os.path.join(open_model_zoo_path, "tools", "downloader", "downloader.py")
downloader_command = [
    "python",
    model_downloader,
    "--output_dir",
    base_model_dir,
    "--jobs",
    "4",
    "--cache_dir",
    omz_cache_dir,
    "--precision",
    precision,
    "--all",
]

print(f"Start downloading models with the following command: \n{' '.join(downloader_command)}")

download_result = subprocess.run(
    downloader_command,
    shell=False,
    check=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True,
)
if download_result.returncode == 0:
    print("Downloading models succeeded. ")
else:
    print(f"Downloading models failed. The error message is: {download_result.stderr}")

## Convert models to OpenVINO IR format

The Model Converter converts Open Model Zoo models to IR format. Models that were originally in that format are ignored. Like the Model Downloader, the Model Converter also supports `--all`, `--name` and `--list` options to specify which models to convert, and `--precisions` to specify that only models with that precision must be converted. Depending on how OpenVINO is installed, it may be necessary to add the path to the Model Optimizer. This can be done with the `--mo` option

In [ ]:
# The code in this cell sets the model_optimizer_path if OpenVINO was installed with default settings. Adjust this if necessary.
if sys.platform.startswith("win"):
    model_optimizer_path = (
        r"C:\Program Files (x86)\intel\openvino_2021\deployment_tools\model_optimizer\mo.py"  # Windows
    )
else:
    model_optimizer_path = "/opt/intel/openvino_2021/deployment_tools/model_optimizer/mo.py"  # Linux/MacOS

In [ ]:
model_converter = os.path.join(open_model_zoo_path, "tools", "downloader", "converter.py")
converter_command = [
    "python",
    model_converter,
    "--download_dir",
    base_model_dir,
    "--precisions",
    precision,
    "--mo",
    model_optimizer_path,
    "--jobs",
    "6",
    "--all",
]

print(f"Start converting models with the following command: \n{' '.join(converter_command)}")


converter_result = subprocess.run(
    converter_command,
#     stdout=subprocess.PIPE,
#     stderr=subprocess.PIPE,
#     capture_output=False,
    shell=False,
)
if converter_result.returncode == 0:
    print("Converting models succeeded.")
else:
    print(
        f"There were some error messages while converting the models. Check `converter_result.stderr` for more details."
    )

## Get model info

The Info Dumper returns information for the Open Model Zoo models. It returns a list of dictionaries with per model:

* model name
* description
* framework that was used to train the model
* license url
* precisions supported by the model
* subdirectory: the location of the downloaded model
* task type

In [ ]:
info_command = os.path.join(open_model_zoo_path, "tools", "downloader", "info_dumper.py")
info_result = subprocess.run(
    [
        "python",
        info_command,
        "--list",
        "models.lst",
        #            "--mo",
        #            "/opt/intel/openvino_2021.2.200/deployment_tools/model_optimizer/mo.py",
    ],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    capture_output=False,
    shell=False,
    text=True,
)
info = json.loads(info_result.stdout)
modelnames = [model["name"] for model in info]

info[0]